In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crowdflower-search-relevance/train.csv.zip
/kaggle/input/crowdflower-search-relevance/sampleSubmission.csv.zip
/kaggle/input/crowdflower-search-relevance/test.csv.zip


In [2]:
train = pd.read_csv('/kaggle/input/crowdflower-search-relevance/train.csv.zip')
test = pd.read_csv('/kaggle/input/crowdflower-search-relevance/test.csv.zip')
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10158 entries, 0 to 10157
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10158 non-null  int64  
 1   query                10158 non-null  object 
 2   product_title        10158 non-null  object 
 3   product_description  7714 non-null   object 
 4   median_relevance     10158 non-null  int64  
 5   relevance_variance   10158 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 476.3+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22513 entries, 0 to 22512
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   22513 non-null  int64 
 1   query                22513 non-null  object
 2   product_title        22513 non-null  object
 3   product_description  17086 non-null  object
dtypes: int64(1), object(3)
memory usage: 703.7+ KB


In [5]:
train.describe()

,id,median_relevance,relevance_variance
count,10158.000000,10158.000000,10158.000000
mean,16353.103071,3.309805,0.377863
std,9447.106683,0.980666,0.389707
min,1.000000,1.000000,0.000000
25%,8078.750000,3.000000,0.000000
50%,16349.500000,4.000000,0.471000
75%,24570.750000,4.000000,0.471000
max,32668.000000,4.000000,1.470000


In [6]:
train['query'].map(lambda x:len(x.split())).value_counts()

2    5379
3    2819
4     925
1     885
6      81
5      69
Name: query, dtype: int64

In [7]:
train['product_title'].map(lambda x:len(x.split())).value_counts()

7     1288
6     1284
8     1183
9     1122
5     1002
10     880
11     744
12     678
4      550
13     453
14     289
15     181
3      174
17      78
16      66
2       51
18      25
19      24
20      15
24      11
21       7
25       7
27       6
26       6
22       6
28       6
23       6
29       4
1        2
46       2
32       2
38       1
44       1
34       1
31       1
43       1
41       1
Name: product_title, dtype: int64

In [8]:
split = int(len(train)*0.8)
train_0, dev = train[:split], train[split:]

In [9]:
clean_train_1 = train_0[train_0.relevance_variance <1].copy()
clean_train_2 = train_0[train_0.relevance_variance <0.50].copy()
dev.describe()

,id,median_relevance,relevance_variance
count,2032.000000,2032.000000,2032.000000
mean,29406.014764,3.319390,0.361364
std,1870.217123,0.972218,0.379619
min,26215.000000,1.000000,0.000000
25%,27777.250000,3.000000,0.000000
50%,29410.000000,4.000000,0.471000
75%,31014.750000,4.000000,0.471000
max,32668.000000,4.000000,1.470000


In [10]:
clean_train_1.describe()

,id,median_relevance,relevance_variance
count,7558.000000,7558.000000,7558.000000
mean,13074.201111,3.344403,0.321038
std,7571.543134,0.974908,0.332482
min,1.000000,1.000000,0.000000
25%,6496.500000,3.000000,0.000000
50%,13129.000000,4.000000,0.471000
75%,19563.250000,4.000000,0.471000
max,26208.000000,4.000000,0.980000


In [11]:
clean_train_2.describe()

,id,median_relevance,relevance_variance
count,6206.000000,6206.000000,6206.000000
mean,13154.066549,3.432646,0.202590
std,7570.559783,0.959901,0.232434
min,1.000000,1.000000,0.000000
25%,6576.750000,3.000000,0.000000
50%,13299.500000,4.000000,0.000000
75%,19708.000000,4.000000,0.471000
max,26208.000000,4.000000,0.490000


In [12]:
## Skipping product description as it's too lengthy and missing values
train = clean_train_1
train_input = train.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
dev_input =  dev.apply(lambda x: x['query']+' '+x['product_title'], axis=1)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(ngram_range=(1, 5),stop_words = 'english', strip_accents='unicode')
train_x = tfidf.fit_transform(train_input)
dev_x = tfidf.transform(dev_input)

In [14]:
print(train_x)

  (0, 111688)	0.19489945150034094
  (0, 9713)	0.19489945150034094
  (0, 46183)	0.19489945150034094
  (0, 134143)	0.19489945150034094
  (0, 25545)	0.17556656718132257
  (0, 72207)	0.19489945150034094
  (0, 111687)	0.19489945150034094
  (0, 9712)	0.19489945150034094
  (0, 46182)	0.19489945150034094
  (0, 134141)	0.17556656718132257
  (0, 25544)	0.17556656718132257
  (0, 47016)	0.19489945150034094
  (0, 72206)	0.19489945150034094
  (0, 111686)	0.19489945150034094
  (0, 9711)	0.19489945150034094
  (0, 46180)	0.17556656718132257
  (0, 134140)	0.17556656718132257
  (0, 25541)	0.13847739604041226
  (0, 122325)	0.16094180476512956
  (0, 47015)	0.19489945150034094
  (0, 72205)	0.18634451390491188
  (0, 111685)	0.19489945150034094
  (0, 9709)	0.17556656718132257
  (0, 46179)	0.17556656718132257
  (0, 134136)	0.13847739604041226
  :	:
  (7557, 59411)	0.18400006198534766
  (7557, 36554)	0.1759235433716615
  (7557, 91262)	0.1759235433716615
  (7557, 110820)	0.17019316222044095
  (7557, 110819)	0.17

adding further feature engineering

In [15]:
train_y, dev_y = train.median_relevance.to_list(), dev.median_relevance.to_list()
train_y = [(x-1)/3 for x in train_y]
dev_y = [(x-1)/3 for x in dev_y]
np.mean(train_y), np.max(train_y), np.min(train_y)

(0.781467760430449, 1.0, 0.0)

In [16]:
from sklearn.metrics import mean_squared_error, cohen_kappa_score, make_scorer
def reg_scorer(true, pred):
    pred = [min(1, max(0,x)) for x in pred]
    pred = [int(round((x*3)+1)) for x in pred]
    true = [int(round((x*3)+1)) for x in true]
    return cohen_kappa_score(true, pred)

In [ ]:
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
#clf = LinearRegression().fit(train_x, train_y)
#clf = SGDRegressor(verbose=1,n_iter_no_change=20).fit(train_x, train_y)
param_grid = {'C': [1,2,5,10], 'epsilon':[0.1,0.2,0.5], 'kernel': ('linear', 'rbf', 'poly','sigmoid')}
svr  = SVR()
scorer = make_scorer(reg_scorer, greater_is_better=True)
clf = GridSearchCV(svr, param_grid, verbose=True,scoring=scorer, n_jobs=-1)
clf.fit(train_x, train_y)
clf.best_estimator_, clf.best_params_, clf.best_score_

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

random_forest = RandomForestClassifier()

random_forest_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf = GridSearchCV(random_forest, random_forest_param_grid, verbose=True,scoring=scorer, n_jobs=-1, cv=5)
rf.fit(train_x, train_y)
rf.best_estimator_, rf.best_params_, rf.best_score_

In [ ]:
## 0.26 is the best score till now

preds = clf.best_estimator_.predict(dev_x)
mean_squared_error(dev_y, preds),  reg_scorer(dev_y, preds)

preds_rf = rf.best_estimator_.predict(dev_x)
mean_squared_error(dev_y, preds_rf),  reg_scorer(dev_y, preds_rf)

In [ ]:
test_input =  test.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
test_x = tfidf.transform(test_input)
pred = clf.best_estimator_.predict(test_x)
pred = [min(1, max(0,x)) for x in pred]
pred = [int(round((x*3)+1)) for x in pred]
out = pd.DataFrame({"id": test.id.to_list(), "prediction": pred})
out.to_csv('submission.csv', index=False)

In [ ]:
test_input =  test.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
test_x = tfidf.transform(test_input)
pred = rf.best_estimator_.predict(test_x)
pred = [min(1, max(0,x)) for x in pred]
pred = [int(round((x*3)+1)) for x in pred]
out = pd.DataFrame({"id": test.id.to_list(), "prediction": pred})
out.to_csv('submission.csv', index=False)

In [ ]:
sub = pd.read_csv('/kaggle/input/crowdflower-search-relevance/sampleSubmission.csv.zip')
sub

In [ ]:
out